In [ ]:
import nest
import os
import anndata
import scipy
import sklearn.metrics
import numpy as np
import pandas as pd
import seaborn as sns
import scanpy as sc
import squidpy as sq
import networkx as nx
from pathlib import Path
import time
from tqdm import tqdm
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from matplotlib import rc
import networkx as nx
import colorcet as cc

import warnings
warnings.filterwarnings("ignore")

# Main Analysis

In [ ]:
dataset = "slideseq"

neighbor_eps, min_samples, hotspot_min_size = nest.data.default_parameters(dataset)

In [ ]:
# Configuration for plotting and saving
dataset_dir = os.path.expanduser("~/Documents/data/")
cache_dir = os.path.expanduser(f"data/{dataset}")
image_save_dir = os.path.expanduser(f"images/{dataset}/")
if not os.path.exists(cache_dir):
    os.makedirs(cache_dir)
if not os.path.exists(image_save_dir):
    os.makedirs(image_save_dir)
nest.plot.set_dataset_plot_parameters(dataset)

def save_fig(fig, name):
    fig.savefig(os.path.join(image_save_dir, name), dpi=300, bbox_inches='tight', transparent=True)

In [ ]:
adata = nest.data.get_data(dataset, dataset_dir, normalize=True)

In [ ]:
%%time
nest.spatial_smoothing(adata)
print(adata.shape)

In [ ]:
%%time
res = nest.compute_gene_hotspots(adata, verbose=True, log=True,
                           eps=neighbor_eps, min_samples=min_samples, min_size=hotspot_min_size)
print(res)

In [ ]:
tmp = nest.get_min_samples_from_density(adata, eps=neighbor_eps, density=0.1)
print(min_samples, tmp)

In [ ]:
%%time
# resolution = 1.0
resolution = 1.0
nest.coexpression_hotspots(adata, threshold=0.35, min_size=30, cutoff=0.5, min_genes=8, resolution=resolution)

In [ ]:
%%time
nest.compute_multi_boundaries(adata, 0.005, 0.00001)

# We get a slightly prettier picture if we fine-tune two of the narrower hotspots but this is purely cosmetic
for idx in [2, 8]:
    boundary = nest.compute_hotspot_boundary(adata, f"hotspots_multi_{idx}", 0, alpha_max=0.015,
                                                alpha_min=0.00001)
    adata.uns["multi_boundaries"][str(idx)] = boundary

In [ ]:
%%time
nest.plot.multi_hotspots(adata, legend_ncol=2)

In [ ]:
len([v for v in adata.obs.columns if "hotspots_multi_" in v])

In [ ]:
#adata.uns['multi_hotspots_colors'] = {int(k): v for k, v in adata.uns['multi_hotspots_colors'].items()}
%matplotlib inline
plt.rcParams['figure.figsize'] = [5, 5]
rc('font',**{'family':'serif','serif':['Arial'], 'size':5})
nest.plot.nested_structure_plot(adata, figsize=(5, 1.5), fontsize=8, legend_ncol=4, alpha_high=0.75, alpha_low=0.15,
                               legend_kwargs={'loc':"upper left", 'bbox_to_anchor':(1, 1.03)})

In [ ]:
nest.geometry_scores(adata)

In [ ]:
# Alternate visualization that makes it easier to see specific coexpression hotspots
%matplotlib inline
plt.rcParams['figure.figsize'] = [5, 5]
rc('font',**{'family':'serif','serif':['Arial'], 'size':5})
sns.set_color_codes()

nest.plot.all_coex_hotspots(adata, figsize=(6.5, 9))
#plt.savefig(os.path.expanduser(f'images/slideseq/all_coex.png'), dpi=300)
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(2, 2))
nest.plot.spatial(adata, color="geometry", color_map="Blues", title="", ax=ax, frameon=False, show=False)
ax.collections[0].colorbar.remove()
#save_fig(fig, "coherence")

In [ ]:
res = nest.geometric_markers(adata, [3, 5, 17, 7, 6])

In [ ]:
# Visualizations of expression of top-3 marker genes for each region in hippocampus

%matplotlib inline
res_sub = {k: v[:3] for k, v in res.items()}
fig, ax = nest.plot.tracks_plot(adata, res_sub, width=2.5, track_height=0.1, fontsize=6, 
                                marked_genes=[])
#save_fig(fig, "tracks_plot.pdf")

In [ ]:
# Examples of spatial expresssion of particular hippocampus hierarchical marker genes

%matplotlib inline
plt.rcParams['figure.figsize'] = [6.5, 8]
rc('font',**{'family':'serif','serif':['Arial'], 'size':7})

fig, axs = plt.subplots(5, 4, figsize=[6.5, 8])
axs = axs.ravel()
gene_list = [a for k, v in res.items() for a in v[:4]]
for idx, gene in enumerate(gene_list):
    ax = axs[idx]
    nest.plot.spatial(adata, color=gene, color_map="Blues", ax=ax, show=False)
    ax.collections[0].colorbar.remove()
    #break
    
#save_fig(fig, "hippo_marker_genes.png")

In [ ]:
# HMRF can be quite slow on this dataset
regions = 10
hmrf = nest.hmrf.HMRFSegmentationModel(adata, regions=regions, k=6, label_name=f"class_hmrf_{regions}", beta=1)
hmrf.fit(max_iterations=50, update_labels=True, verbose=True)
nest.write(adata, os.path.join(cache_dir, "adata.h5ad"))

fig, ax = plt.subplots(figsize=(5, 5))
nest.plot.spatial(adata, color="class_hmrf_10", ax=ax, frameon=False, title="")

# Panels for Fig 1

In [ ]:
# Plot of the dataset highlighting the hippocampus
%matplotlib inline
rc('font',**{'family':'serif','serif':['Arial'], 'size':7})
nest.plot.spatial(adata, color="class")

In [ ]:
# Fig. 1a
%matplotlib inline
rc('font',**{'family':'serif','serif':['Arial'], 'size':5})
fig, ax = plt.subplots(figsize=(2, 2))
nest.plot.multi_hotspots(adata, inds=[3], alpha=0, base_color="class", linewidth=1, ax=ax, title="")
save_fig(fig, "hippocampal_cell_type.pdf")

In [ ]:
#plot the marker genes
fig, axs = plt.subplots(1, 4, figsize=(5, 2))
genes = ["Nrgn", "Neurod6", "Wfs1", "Chgb", "C1ql2"]
vmaxs = [0.5, 0.5, 1.0, 0.5]
for idx, (gene, vmax) in enumerate(zip(genes, vmaxs)):
    nest.plot.spatial(adata, color=gene, color_map="Blues", show=False, ax=axs[idx], frameon=False, vmax=vmax,
                     title="")
    axs[idx].collections[0].colorbar.remove()

In [ ]:
# Showing the network - use the full hippocampus and the dentate structure
full_genes = ["Nrgn", "Lppr4", "Rbfox3", "St6galnac5"]
dentate_genes = ["C1ql2", "Stxbp6", "Fam163b", "Lrrtm4"]
hippocampus_bbox = [-1900, 2410, -850, 950]
for gene in [*full_genes, *dentate_genes]:
    fig, ax = plt.subplots(figsize=(1,1))
    if gene in full_genes:
        palette = [adata.uns['multi_hotspots_colors']['3']]
    else:
        palette = [adata.uns['multi_hotspots_colors']['8']]
    nest.plot.hotspots(adata, color=gene, groups=[1], labels=False, na_color=[1.0, 1.0, 1.0, 1.0],
                      crop_coord=hippocampus_bbox, frameon=False, title="", show=False, ax=ax,
                      palette=palette)
    save_fig(fig, f"main_hotspot_{gene}.pdf")

In [ ]:
ids = [3, 5, 7, 8, 15]
genes = [f"multi_{k}" for k in ids]
hippocampus_bbox = [-1900, 2410, -850, 950]
for k, gene in zip(ids, genes):
    fig, ax = plt.subplots(figsize=(1,1))
    nest.plot.hotspots(adata, color=gene, groups=[0], labels=False, na_color=[1.0, 1.0, 1.0, 0.0],
                      crop_coord=hippocampus_bbox, frameon=False, title="", show=False, ax=ax,
                      palette=[adata.uns['multi_hotspots_colors'][str(k)]])
    save_fig(fig, f"main_hotspot_{gene}.pdf")

In [ ]:
fig, ax = plt.subplots(figsize=(3.2, 3.2))
nest.plot.multi_hotspots(adata, ax=ax, show=False)
save_fig(fig, "all_coex.pdf")

In [ ]:
fig, ax = plt.subplots(figsize=(1.0, 1.0))
nest.plot.spatial(adata, color="Nrgn", ax=ax, show=False, title="", color_map="Blues", frameon=False)
ax.collections[0].colorbar.remove()
save_fig(fig, "Nrgn_expr.pdf")

from nest.hotspot.hotspot import _compute_cutoff
v = adata[:, 'Nrgn'].X.toarray()
cutoff = _compute_cutoff(v, log=True)
fig, ax = plt.subplots(figsize=(1.0, 1.0))
nest.plot.spatial(adata, color="Nrgn", ax=ax, show=False, title="", color_map="Blues", frameon=False,
                 vmin=cutoff-1e-6, vmax=cutoff+1e-6)
ax.collections[0].colorbar.remove()
save_fig(fig, "Nrgn_threshold.pdf")

fig, ax = plt.subplots(figsize=(1.0, 1.0))
cc = adata.uns['multi_hotspots_colors']['3']
nest.plot.hotspots(adata, "Nrgn", ax=ax, show=False, title="", frameon=False, palette={1: cc, 2: 'C1'})
save_fig(fig, "Nrgn_hotspots.pdf")

In [ ]:
gene = "Stxbp6"
fig, ax = plt.subplots(figsize=(1.0, 1.0))
cc = adata.uns['multi_hotspots_colors']['8']
nest.plot.hotspots(adata, gene, ax=ax, show=False, title="", frameon=False, 
                  palette={1: cc, 2:'C0', 3: 'C1', 4: 'C2', 5: 'C3', 6: 'C4'})
save_fig(fig, f"{gene}_hotspots.pdf")

In [ ]:
for k in [1, 2]:
    fig, ax = plt.subplots(figsize=(1,1))
    nest.plot.hotspots(adata, color="Nrgn", groups=[k], labels=False, na_color=[1.0, 1.0, 1.0, 1.0],
                      frameon=False, title="", show=False, ax=ax)
    save_fig(fig, f"Nrgn_hotspot_{k}.pdf")